Neste notebook, vamos codificar Redes Neurais de forma manual para tentar entender intuitivamente como elas são implementadas na prática.

# Sumário

- [Exemplo 1](#Exemplo-1)
- [Exemplo 2](#Exemplo-2)
- [O que precisamos para implementar uma Rede Neural?](#O-que-precisamos-para-implementar-uma-Rede-Neural?)
- [Referências](#Referências)

# Imports e Configurações

In [1]:
import numpy as np

# Exemplo 1

<img src='images/backprop_example_1.png'>

In [2]:
def sigmoid(x, derivative=False):
    if derivative:
        y = sigmoid(x)
        return y*(1-y)
    return 1.0/(1.0 + np.exp(-x))

In [3]:
x = np.array([[0.05, 0.10]])
y = np.array([[0.01, 0.99]])

w1 = np.array([[0.15, 0.20], [0.25, 0.30]])
b1 = np.array([[0.35]])
w2 = np.array([[0.40, 0.45], [0.50, 0.55]])
b2 = np.array([[0.60]])

learning_rate = 0.5

for i in range(1):
    # feed-forward
    # 1a camada
    inp1 = np.dot(x, w1.T) + b1
    h1 = sigmoid(inp1)
    
    # 2a camada
    inp2 = np.dot(h1, w2.T) + b2
    out = sigmoid(inp2) 
    
    cost = 0.5 * np.sum((y - out)**2)
    
    # backpropagation
    dout = -(y - out)

    # 2a camada
    dinp2 = sigmoid(inp2, derivative=True)*dout
    dh1 = np.dot(dinp2, w2)
    dw2 = np.dot(dinp2.T, h1)
    db2 = 1.0*dinp2.sum(axis=0, keepdims=True)
    
    # 1a camada
    dinp1 = sigmoid(inp1, derivative=True)*dh1
    dx = np.dot(dinp1, w1)
    dw1 = np.dot(dinp1.T, x)
    db1 = 1.0*dinp1.sum(axis=0, keepdims=True)

    print(dw1)
    print(dw2)

    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1

    print(w1)
    print(w2)

[[0.00043857 0.00087714]
 [0.00049771 0.00099543]]
[[ 0.08216704  0.08266763]
 [-0.02260254 -0.02274024]]
[[0.14978072 0.19956143]
 [0.24975114 0.29950229]]
[[0.35891648 0.40866619]
 [0.51130127 0.56137012]]


# Exemplo 2

In [4]:
def linear(x, derivative=False):
    return np.ones_like(x) if derivative else x

def relu(x, derivative=False):
    if derivative:
        x = np.where(x <= 0, 0, 1)
    return np.maximum(0, x)

def softmax(x, y_oh=None, derivative=False):
    if derivative:
        y_pred = softmax(x)
        k = np.nonzero(y_pred * y_oh)
        pk = y_pred[k]
        y_pred[k] = pk * (1.0 - pk)
        return y_pred
    exp = np.exp(x)
    return exp / np.sum(exp, axis=1, keepdims=True)

def neg_log_likelihood(y_oh, y_pred, derivative=False):
    k = np.nonzero(y_pred * y_oh)
    pk = y_pred[k]
    if derivative:
        y_pred[k] = (-1.0 / pk)
        return y_pred
    return np.mean(-np.log(pk))

def softmax_neg_log_likelihood(y_oh, y_pred, derivative=False):
    y_softmax = softmax(y_pred)
    if derivative:
        k = np.nonzero(y_pred * y_oh)
        dlog = neg_log_likelihood(y_oh, y_softmax, derivative=True)
        dsoftmax = softmax(y_pred, y_oh, derivative=True)
        y_softmax[k] = dlog[k] * dsoftmax[k]
        return y_softmax / y_softmax.shape[0]
    return neg_log_likelihood(y_oh, y_softmax)

In [5]:
x = np.array([[0.1, 0.2, 0.7]])
y = np.array([[1, 0, 0]])
w1 = np.array([[0.1, 0.2, 0.3], [0.3, 0.2, 0.7], [0.4, 0.3, 0.9]])
b1 = np.ones((1,3))
w2 = np.array([[0.2, 0.3, 0.5], [0.3, 0.5, 0.7], [0.6, 0.4, 0.8]])
b2 = np.ones((1,3))
w3 = np.array([[0.1, 0.4, 0.8], [0.3, 0.7, 0.2], [0.5, 0.2, 0.9]])
b3 = np.ones((1,3))

learning_rate = 0.01

N, D = x.shape

for i in range(301):
    # feedforward
    # 1a camada
    inp1 = np.dot(x, w1.T) + b1 # [1x3]x[3x3] = [1x3]
    h1 = relu(inp1) # [1x3]
    
    # 2a camada
    inp2 = np.dot(h1, w2.T) + b2 # [1x3]x[3x3] = [1x3]
    h2 = sigmoid(inp2) # [1x3]
    
    # 3a camada
    inp3 = np.dot(h2, w3.T) + b3 # [1x3]x[3x3] = [1x3]
    out = linear(inp3) # [1x3]
    
    cost = softmax_neg_log_likelihood(y, out) # [1x3]
    
    # backpropagation
    dout = softmax_neg_log_likelihood(y, out, derivative=True) # [1x3]

    # 3a camada
    dinp3 = linear(inp3, derivative=True)*dout # [1x3]
    dh2 = np.dot(dinp3, w3) # [1x3]x[3x3] = [1x3]
    dw3 = np.dot(dinp3.T, h2) # [3x1]x[1x3] = [3x3]
    db3 = 1.0*dinp3.sum(axis=0, keepdims=True) # [1x3]
    
    # 2a camada
    dinp2 = sigmoid(inp2, derivative=True)*dh2 # [1x3]
    dh1 = np.dot(dinp2, w2) # [1x3]x[3x3] = [1x3]
    dw2 = np.dot(dinp2.T, h1) # [3x1]x[1x3] = [3x3]
    db2 = 1.0*dinp2.sum(axis=0, keepdims=True)
    
    # 1a camada
    dinp1 = relu(inp1, derivative=True)*dh1 # [1x3]
    dx = np.dot(dinp1, w1) # [1x3]x[3x3] = [1x3]
    dw1 = np.dot(dinp1.T, x) # [3x1]x[1x3] = [3x3]
    db1 = 1*dinp1.sum(axis=0, keepdims=True)

    w3 = w3 - learning_rate*dw3
    b3 = b3 - learning_rate*db3
    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1

    if i % 30 == 0:
        cost = softmax_neg_log_likelihood(y, out)
        print(cost)

for w in [w1, w2, w3]:
    print(w)

1.1674456052871238
0.607979582043569
0.36558760028370507
0.24943451548999276
0.18547915491555542
0.14614614804167658
0.1199051550833629
0.10131084011234358
0.08751844300750028
0.07691720650783256
0.06853445083252653
[[0.10083595 0.2016719  0.30585165]
 [0.30086971 0.20173942 0.70608796]
 [0.40145052 0.30290104 0.91015363]]
[[0.20544723 0.30673159 0.50749567]
 [0.30994562 0.5123005  0.71366784]
 [0.61065514 0.41317913 0.81464085]]
[[ 0.66465527  0.98758148  1.39393956]
 [ 0.05020341  0.44006253 -0.06274803]
 [ 0.18514132 -0.12764401  0.56880846]]


# O que precisamos para implementar uma Rede Neural?

- Armazenar as entradas ($inp1, inp2,\dots$) e as saídas ($h1, h2, \dots$) das funções de ativação de cada camada
- A atualização dos pesos é efetuada somente após o cálculo de todos os gradientes
- Sempre que a gente calcula um gradiente, a gente multiplica o resultado pelo último gradiente calculado (__regra da cadeia__)

# Referências

- [Neural Network from Scratch](https://beckernick.github.io/neural-network-scratch/)
- [Backpropagation Algorithm](https://theclevermachine.wordpress.com/tag/backpropagation-algorithm/)
- [Back-Propagation is very simple. Who made it Complicated ?](https://becominghuman.ai/back-propagation-is-very-simple-who-made-it-complicated-97b794c97e5c)
- [A Step by Step Backpropagation Example](https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/)
- [Understanding softmax and the negative log-likelihood](https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/)